# Requirements

In [1]:
!pip install --user gdelt

# Import modules

In [239]:
from datetime import date, timedelta
from gdelt import gdelt as gdelt_client
import matplotlib.pyplot as plot
import seaborn
import tempfile

# Query the knowledge graph
Use coverage option for querying all daily records. Otherwise records collected from the last 15 minutes are returned.
Use the date option to filter by date.

In [240]:
def get_graph(date, coverage=False):
    client = gdelt_client(version=2)
    events = client.Search(date.strftime("%Y %m %d"), table="gkg", coverage=coverage)
    del client
    return events

def get_today_graph(coverage=False):
    return get_graph(date.today(), coverage)

def get_yesterday_graph(coverage=False):
    return get_graph(date.today()-timedelta(days=1), coverage)

In [241]:
date = date.today()-timedelta(days=1)
graph = get_graph(date, coverage=True)
corona_records = graph.loc[graph["V2Themes"].str.contains("TAX_DISEASE_CORONAVIRUS", na=False)]

# Save records to temp

In [242]:
#corona_records.to_csv("{}/corona_{}.csv".format(tempfile.gettempdir(), date.strftime("%Y%m%d")), index=False)

# Location type exploding and filtering

In [243]:
from enum import Enum

class location_type(Enum):
    """Location type
        Defines the different location types.
    """
    UNKNOWN = 0
    COUNTRY = 1
    USSTATE = 2
    USCITY = 3
    WORLDCITY = 4
    WORLDSTATE = 5

class gdelt_location:
    """GDELT location
        Defines a GDELT location.
    """
    def __init__(self, location_typeid=0, name=None, country_code=None, admin1_code=None, lat=None, lon=None, feature_id=None):        
        self.location_type = location_type(int(location_typeid))
        self.location_name = name
        self.country_code = country_code
        self.admin1_code = admin1_code
        self.location_lat = lat
        self.location_lon = lon
        self.feature_id = feature_id
        
    def has_location_type(self, location_type):
        return location_type == self.location_type
    
    def location_type_matches(self, location_types):
        return self.location_type in location_types
    
    def __str__(self):
        return self.location_name
    
class location_filter():
    """Location Filter
        Defines different filters which can be applied on the dataframes.
    """
    def filter_by_type(self, gkg_dataframe, location_type):
        return gkg_dataframe.loc[gkg_dataframe.apply(lambda record: record["GDELT_Locations"].has_location_type(location_type), axis=1)]
    
    def filter_by_types(self, gkg_dataframe, location_types):
        return gkg_dataframe.loc[gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_type_matches(location_types), axis=1)]

def split_location_entries(locations):
    return [gdelt_location(*location) if 7 == len(location) else gdelt_location() for location in locations]

def split_locations(record):
    return split_location_entries([location.split("#") for location in str(record["Locations"]).split(";")])


# Filter the locations by location type

In [244]:
corona_locations = corona_records.copy(deep=True)
corona_locations["GDELT_Locations"] = corona_records.apply(lambda record: split_locations(record), axis=1)
corona_locations_exploded = corona_locations.explode("GDELT_Locations")

filter = location_filter()
corona_filtered_locations = filter.filter_by_types(corona_locations_exploded, [location_type.WORLDCITY, location_type.USCITY])
del corona_records
del graph

# Extract the coordinates and the name from the GDELT location

In [245]:
def to_point_locations(gkg_dataframe):
    point_locations = corona_filtered_locations[["GKGRECORDID", 
                                                 "DATE", 
                                                 "SourceCollectionIdentifier",
                                                 "SourceCommonName",
                                                 "DocumentIdentifier",
                                                 "GDELT_Locations"]].copy(deep=True)
    point_locations["Location_Name"] = gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_name, axis=1)
    point_locations["Location_Lat"] = gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_lat, axis=1)
    point_locations["Location_Lon"] = gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_lon, axis=1)
    return point_locations.drop("GDELT_Locations", axis=1)

corona_point_locations = to_point_locations(corona_filtered_locations)
del corona_filtered_locations

corona_point_locations

,GKGRECORDID,DATE,SourceCollectionIdentifier,SourceCommonName,DocumentIdentifier,Location_Name,Location_Lat,Location_Lon
0,20200313000000-0,20200313000000,1,kawc.org,https://www.kawc.org/post/performing-arts-and-...,"Carnegie Hall, New York, United States",40.7651,-73.9799
0,20200313000000-0,20200313000000,1,kawc.org,https://www.kawc.org/post/performing-arts-and-...,"Golden Gate Park, California, United States",37.7691,-122.48
0,20200313000000-0,20200313000000,1,kawc.org,https://www.kawc.org/post/performing-arts-and-...,"Philadelphia, Pennsylvania, United States",39.9523,-75.1638
0,20200313000000-0,20200313000000,1,kawc.org,https://www.kawc.org/post/performing-arts-and-...,"Los Angeles, California, United States",34.0522,-118.244
0,20200313000000-0,20200313000000,1,kawc.org,https://www.kawc.org/post/performing-arts-and-...,"Metropolitan Museum Of Art, New York, United S...",40.7795,-73.9629
...,...,...,...,...,...,...,...,...
192764,20200313224500-2310,20200313224500,1,martinsvillebulletin.com,https://www.martinsvillebulletin.com/opinion/e...,"Henry County, Virginia, United States",36.6668,-79.8831
192764,20200313224500-2310,20200313224500,1,martinsvillebulletin.com,https://www.martinsvillebulletin.com/opinion/e...,"Martinsville, Virginia, United States",36.6915,-79.8725
192768,20200313224500-2314,20200313224500,1,kaok.com,https://www.kaok.com/news/orlando-bloom-coming...,"Hollywood, California, United States",34.0983,-118.327
192768,20200313224500-2314,20200313224500,1,kaok.com,https://www.kaok.com/news/orlando-bloom-coming...,"Prague, Praha, HlavníEsto, Czech Republic",50.0833,14.4667


# Save point locations to temp

In [246]:
corona_point_locations.to_csv("{}/corona_locations_{}.csv".format(tempfile.gettempdir(), date.strftime("%Y%m%d")), index=False)